In [1]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification

In [2]:
data = pd.read_csv('../Anomaly-Detection/amazon_reviews.txt', delimiter="\t")
features = ['RATING', 'REVIEW_TEXT', 'VERIFIED_PURCHASE', 'LABEL']


#data_true = data[  data['LABEL'] == '__label1__' ]

data_shortened = data[features]

data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('N', 0)
data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('Y', 1)

data_shortened['LABEL'] = data_shortened['LABEL'].replace('__label1__', 1)
data_shortened['LABEL'] = data_shortened['LABEL'].replace('__label2__', 0)

#data_pos = data_shortened[  data_shortened['RATING'] == 5 ]
#data_neg = data_shortened[data_shortened['RATING'] == 1]

#print(data_pos.head())

labels = data_shortened.pop('LABEL')

#print(data_pos.head())

#print(data_pos.dtypes)

<ipython-input-2-5d0a6fe65350>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('N', 0)
<ipython-input-2-5d0a6fe65350>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('Y', 1)
<ipython-input-2-5d0a6fe65350>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [4]:

import numpy as np

reviews_array = data_shortened["REVIEW_TEXT"].to_numpy(dtype=object)
labels_array = labels.to_numpy(dtype=np.int32)

#Shuffling the dataset
data_length = len(reviews_array)
idx = np.random.permutation(data_length)
shuffled_reviews_array = reviews_array[idx]
shuffled_labels_array = labels_array[idx]

# Train-test split
train_ratio = 0.8
train_data_len = int( train_ratio * data_length )

train_reviews_array = shuffled_reviews_array[:train_data_len]
train_labels_array = shuffled_labels_array[:train_data_len]

test_reviews_array = shuffled_reviews_array[train_data_len:]
test_labels_array = shuffled_labels_array[train_data_len:]

model_name = 'distilbert-base-uncased' # or bert-base-cased

tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenized_df = tokenizer(data_shortened.head()["REVIEW_TEXT"].tolist(), padding="max_length", truncation=True)

train_reviews = train_reviews_array.tolist()
test_reviews = test_reviews_array.tolist()

#! Take Care
training_size = 200

train_reviews_small = train_reviews[0:training_size]
train_labels_array_small = train_labels_array[0:training_size]
test_reviews_small = test_reviews[0:training_size]
test_labels_array_small = test_labels_array[0:training_size]


train_tokenized_df = tokenizer(train_reviews_small, padding="max_length", truncation=True)
test_tokenized_df = tokenizer(test_reviews_small, padding="max_length", truncation=True)

#print( np.sum(train_labels_array == 0))
#print(train_labels_array[0:5])

#print(type(tokenized_df))
#print(tokenized_df.keys())

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 11.2kB/s]
Downloading: 100%|██████████| 442/442 [00:00<00:00, 221kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 1.43MB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 1.50MB/s]


In [5]:

#Create the input dictionary for the model
train_features = {x: tf.convert_to_tensor(train_tokenized_df[x]) for x in tokenizer.model_input_names}
test_features = {x: tf.convert_to_tensor(test_tokenized_df[x]) for x in tokenizer.model_input_names} 
#print(train_features)

#The dataset now is a tuple of dictionary containint inpud_ids, token_type_ids. attention masks        and  a tf tensor containing the value for the label

#train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, labels.head()))
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels_array_small))
train_tf_dataset = train_tf_dataset.shuffle(training_size).batch(8) #train_data_len

print(train_tf_dataset)

test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels_array_small))
test_tf_dataset = test_tf_dataset.batch(8)

print(test_labels_array_small.dtype)
print(train_labels_array_small.dtype)




<BatchDataset shapes: ({input_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>
int32
int32


In [6]:
import tensorflow.keras as keras
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="accuracy", patience=15,
                                    verbose=1, mode="min", restore_best_weights=True),
        keras.callbacks.ModelCheckpoint(filepath="transformer_model/model_distillbert_200.hdf5", verbose=1, save_best_only=True)
    ]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
    
)

model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=10, verbose=1, callbacks = callbacks) #3 epochs

model.save_pretrained("transformer_model")

Downloading: 100%|██████████| 363M/363M [00:10<00:00, 35.4MB/s]
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should pro

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (

In [13]:

#print(data_pos.head()["REVIEW_TEXT"].tolist()[0])

print(train_labels_array[200:205])

for i in range(200, 205):
    #y = tokenizer(data_shortened.head()["REVIEW_TEXT"].tolist()[i], padding="max_length", truncation=True)
    y = tokenizer(train_reviews[i], padding="max_length", truncation=True)
    y['input_ids'] = tf.convert_to_tensor( [y['input_ids']] )
    if model_name == 'bert-base-cased':
        y['token_type_ids'] = tf.convert_to_tensor( [y['token_type_ids']] )
    y['attention_mask'] = tf.convert_to_tensor( [y['attention_mask']] )

    label = model(y)
    probs = tf.nn.softmax(label.logits)
    print(probs)



[1 1 0 1 0]
tf.Tensor([[0.99846727 0.00153272]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.35731387 0.6426861 ]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.00899104 0.99100894]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.00801229 0.9919877 ]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.00160082 0.9983992 ]], shape=(1, 2), dtype=float32)
